In [6]:
import pandas as pd
from youtube_transcript_api import YouTubeTranscriptApi
import time



In [7]:
df = pd.read_csv("C:\\projects\\NEW AI QUERY TUBE\\AI-QueryTube\\data\\processed\\youtube_metadata_cleaned.csv") 
df.head()


,video_id,title,published_date
0,Vtckgz38QHs,Learn Quick Sort in 13 minutes ⚡,2021-08-13 14:11:31+00:00
1,iLZOL-hmr7M,C format specifiers 🔧,2021-08-15 15:27:58+00:00
2,Hh_se2Zqsdk,Learn HTML images in 3 minutes 📷,2021-08-15 15:28:16+00:00
3,9E3I1URu6fc,C arithmetic operators ➗,2021-08-19 15:00:33+00:00
4,SZfoOAv9tEo,Learn HTML text formatting in 2 minutes 💬,2021-08-21 15:06:21+00:00


In [8]:
df["video_id"] = (
    df["video_id"]
    .astype(str)
    .str.strip()
    .str.replace(r"[^\w\-]", "", regex=True)
)


In [9]:
ytt_api = YouTubeTranscriptApi()

# If resuming, don't overwrite existing transcripts
if "transcript" not in df.columns:
    df["transcript"] = None

failed_videos = []


In [10]:
import time
LIMIT = 10       # change to 10 / 20 only
DELAY = 1.5         # seconds (important)

processed = 0

for idx, row in df.iterrows():

    if processed >= LIMIT:
        break

    # Skip already processed rows
    if pd.notna(row["transcript"]):
        continue

    video_id = row["video_id"]

    try:
        transcript_object = ytt_api.fetch(video_id)

        full_transcript = " ".join(
            snippet.text for snippet in transcript_object.snippets
        )

        df.at[idx, "transcript"] = full_transcript
        print(f"✅ Success: {video_id}")

    except Exception as e:
        print(f"❌ Failed: {video_id} | {type(e).__name__}")
        df.at[idx, "transcript"] = None
        failed_videos.append(video_id)

    processed += 1
    time.sleep(DELAY)




✅ Success: Vtckgz38QHs
✅ Success: iLZOL-hmr7M
✅ Success: Hh_se2Zqsdk
✅ Success: 9E3I1URu6fc
✅ Success: SZfoOAv9tEo
✅ Success: -kXZvKxs9oA
✅ Success: REtiFfgj8SU
✅ Success: 5mqHzWIzruM
✅ Success: 6IQz_cF-Slk
✅ Success: yHX-UwAnoqk


In [11]:
df[["video_id", "transcript"]].head(10)


,video_id,transcript
0,Vtckgz38QHs,hey uh it's you bro hope you're doing well and...
1,iLZOL-hmr7M,all right welcome back everybody in this video...
2,Hh_se2Zqsdk,all right people here's how to add an image to...
3,9E3I1URu6fc,well well well welcome back so we need to talk...
4,SZfoOAv9tEo,all right welcome back everybody so in this vi...
5,-kXZvKxs9oA,hey welcome back in this video i'm going to sh...
6,REtiFfgj8SU,hey everyone here's a quick program that we ca...
7,5mqHzWIzruM,all right welcome back everybody in this video...
8,6IQz_cF-Slk,"Hey. Uh, what's going on everybody? In this vi..."
9,yHX-UwAnoqk,hey welcome back in this video i'm going to ex...


In [12]:
print("Total videos:", len(df))
print("Transcripts fetched:", df["transcript"].notna().sum())


Total videos: 168
Transcripts fetched: 10


In [ ]:
df.to_csv(
    "C:\\projects\\NEW AI QUERY TUBE\\AI-QueryTube\\data\\processed\\videos_with_transcripts.csv",
    index=False
)

